##hash function

In [10]:
import math
import time
import itertools
import numpy as np
start = time.time()

def hash(element1,element2,num_trans):
  result = (element1 + element2) % num_trans
  return result

##Load Data && split dataset into 4 subsets && generate C1 and pairs for each subset

In [11]:
candidates_1 = {}
candidates_2 = {}
candidates_3 = {}
candidates_4 = {}

buckets_1 = {}
buckets_2 = {}
buckets_3 = {}
buckets_4 = {}


num_trans = 0
all_data = []
#read all basket from file
with open('retail.txt', 'r') as file:
  for line in file:
    num_trans += 1
    line_list = [int(x) for x in line.strip().split()]
    all_data.append(line_list)
print(len(all_data))

#slice dataset into 4 parts
data_partition_1 = all_data[0:int(num_trans/4)]
num_trans_1 = len(data_partition_1)

data_partition_2 = all_data[int(num_trans/4):int(num_trans*(2/4))]
num_trans_2 = len(data_partition_2)

data_partition_3 = all_data[int(num_trans*2/4):int(num_trans*3/4)]
num_trans_3 = len(data_partition_3)

data_partition_4 = all_data[int(num_trans*3/4):num_trans]
num_trans_4 = len(data_partition_4)

def count_data(data_partition,candidates,buckets):
    for line in data_partition:
      for item in line:
        if item in candidates.keys():
          candidates.update({item:(candidates[item]+1)})
        else:
          candidates.update({item:1})
      pairs = list(itertools.combinations(line_list, 2))
      for pair in pairs:
        index = hash(pair[0], pair[1],num_trans)
        if index in buckets.keys():
          buckets.update({index:(buckets[index]+1)})
        else:
          buckets.update({index:1})
    return candidates,buckets
#netflix.data retail.txt

# get transactions,c1,buckets
c1_1,buckets_1 = count_data(data_partition_1,candidates_1,buckets_1)
c1_2,buckets_2 = count_data(data_partition_2,candidates_2,buckets_2)
c1_3,buckets_3 = count_data(data_partition_3,candidates_3,buckets_3)
c1_4,buckets_4 = count_data(data_partition_2,candidates_2,buckets_2)
min_support = 0.02
#0.01

88162


##Generate Bitmap


In [12]:
def generate_Bitmap(hashtable,num_trans,min_support):
  bitmap = [0]*num_trans
  for bucket,bucket_count in hashtable.items():
    if bucket_count/num_trans >= min_support:
      bitmap[bucket] = 1
  return bitmap

##Generate L1

In [13]:
def get_L(candidates,num_trans,min_support):
  L = []
  for key,value in candidates.items():
    if value/num_trans >= min_support:
      L.append(key)
  return L

L1_1 = get_L(c1_1,num_trans_1,min_support)
L1_2 = get_L(c1_2,num_trans_2,min_support)
L1_3 = get_L(c1_3,num_trans_3,min_support)
L1_4 = get_L(c1_4,num_trans_4,min_support)

##Prune Dataset

In [14]:
def prune_dataset(candidates,transactions):
  pruned_dataset = []
  for basket in transactions:
    for item in basket:
      if item in candidates:
        pruned_dataset.append(basket)
        break
  return pruned_dataset

pruned_dataset_1 = prune_dataset(L1_1,data_partition_1)
pruned_dataset_2 = prune_dataset(L1_2,data_partition_2)
pruned_dataset_3 = prune_dataset(L1_3,data_partition_3)
pruned_dataset_4 = prune_dataset(L1_4,data_partition_4)

##Count initially_pruned C2

In [15]:
def get_C2(candidates,dataset):
  for basket in dataset:
    for key in candidates.keys():
      if set(key).issubset(set(basket)):
        candidates.update({key:candidates[key]+1})
  return candidates

##Generate C2 && bitmap

In [16]:
C2_1 = list(itertools.combinations(L1_1, 2))
C2_2 = list(itertools.combinations(L1_2, 2))
C2_3 = list(itertools.combinations(L1_3, 2))
C2_4 = list(itertools.combinations(L1_4, 2))

bitmap_1 = generate_Bitmap(buckets_1,num_trans_1,min_support)
bitmap_2 = generate_Bitmap(buckets_2,num_trans_2,min_support)
bitmap_3 = generate_Bitmap(buckets_3,num_trans_3,min_support)
bitmap_4 = generate_Bitmap(buckets_4,num_trans_4,min_support)

# print(bitmap)
# count = 0
# for pair in C2:
#   if hash(pair[0],pair[1]) in buckets.keys():
#     count += 1


##Generate L2

In [ ]:
from ast import And
def get_L2(C2,bitmap,num_trans,pruned_dataset,min_support):
  for pair in C2:
    index = hash(pair[0], pair[1],num_trans)
    if bitmap[index] != 1:
      C2.remove(pair)

  if len(C2) == 0:
    return None
  else:
    initial_value = [0]*len(C2)
    C2_initial = dict(zip(C2, initial_value))
    C2_initial = get_C2(C2_initial,pruned_dataset)
    L2 = get_L(C2_initial,num_trans,min_support)
    return L2

L2_1 = get_L2(C2_1,bitmap_1,num_trans_1,pruned_dataset_1,min_support)
L2_2 = get_L2(C2_2,bitmap_2,num_trans_2,pruned_dataset_2,min_support)
L2_3 = get_L2(C2_3,bitmap_3,num_trans_3,pruned_dataset_3,min_support)
L2_4 = get_L2(C2_4,bitmap_4,num_trans_4,pruned_dataset_4,min_support)

L2_union = set(L2_1).union(set(L2_2)).union(set(L2_3)).union(set(L2_4))
L2_list = list(L2_union)
L2_list_2 = []


for i in range(len(L2_list)):
  flag = False
  for j in range(i+1,len(L2_list)):
    if L2_list[i][0] == L2_list[j][1] and L2_list[i][1] == L2_list[j][0]:
      flag = True
  if flag == False:
    L2_list_2.append(L2_list[i])

def verify_L2(L2_union,num_trans,min_support):
   initial_value = [0]*len(L2_union)
   C2_union = dict(zip(L2_union, initial_value))
   C2_union = get_C2(C2_union,all_data)
   return get_L(C2_union,num_trans,min_support)


L2 = verify_L2(L2_list_2,num_trans,min_support)
print(len(L2))
print(L2)
print("--- seconds---" , (time.time() - start))